<a href="https://colab.research.google.com/github/adnanzaidi548/GridForecast-LSTM/blob/main/Copy_of_GridForecast_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Load CSV dataset from raw GitHub link
import pandas as pd

# GitHub raw link to the dataset
url = 'https://raw.githubusercontent.com/adnanzaidi548/GridForecast-LSTM/main/GridForecast_LSTM_Synthetic.csv'

# Load data with Timestamp parsing
df = pd.read_csv(url, parse_dates=['Timestamp'])

# Print dataset shape and preview first few rows
print("✅ Data loaded successfully. Shape:", df.shape)
display(df.head())



✅ Data loaded successfully. Shape: (8760, 6)


,Timestamp,Voltage (Vt),Load (Pt),Temperature (Tt),Humidity (Ht),Wind Speed (Wt)
0,2024-01-01 00:00:00,237.483571,19.857062,11.805249,81.612424,11.188741
1,2024-01-01 01:00:00,234.308678,8.523707,20.806853,75.574796,21.880177
2,2024-01-01 02:00:00,238.238443,9.062234,-2.878793,68.770123,1.918259
3,2024-01-01 03:00:00,242.615149,17.894549,35.432737,72.757721,3.507044
4,2024-01-01 04:00:00,233.829233,14.678312,-20.707503,63.603447,12.015647


In [ ]:
# Use raw GitHub link for loading CSV
url = 'https://raw.githubusercontent.com/adnanzaidi548/GridForecast-LSTM/main/GridForecast_LSTM_Synthetic.csv'
df = pd.read_csv(url, parse_dates=['Timestamp'])

print("Data loaded:", df.shape)
df.head()


Data loaded: (8760, 6)


,Timestamp,Voltage (Vt),Load (Pt),Temperature (Tt),Humidity (Ht),Wind Speed (Wt)
0,2024-01-01 00:00:00,237.483571,19.857062,11.805249,81.612424,11.188741
1,2024-01-01 01:00:00,234.308678,8.523707,20.806853,75.574796,21.880177
2,2024-01-01 02:00:00,238.238443,9.062234,-2.878793,68.770123,1.918259
3,2024-01-01 03:00:00,242.615149,17.894549,35.432737,72.757721,3.507044
4,2024-01-01 04:00:00,233.829233,14.678312,-20.707503,63.603447,12.015647


In [ ]:
# Interpolate missing values if any
df.interpolate(method='linear', inplace=True)

# Remove outliers using IQR
for col in ['Voltage (Vt)', 'Load (Pt)', 'Temperature (Tt)', 'Humidity (Ht)', 'Wind Speed (Wt)']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)]

df.reset_index(drop=True, inplace=True)
print("After outlier removal:", df.shape)


After outlier removal: (8639, 6)


In [ ]:
# Step: Normalize selected features using Min-Max Scaling
from sklearn.preprocessing import MinMaxScaler

features = ['Voltage (Vt)', 'Load (Pt)', 'Temperature (Tt)', 'Humidity (Ht)', 'Wind Speed (Wt)']
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

# Preview the normalized data
df.head()


,Timestamp,Voltage (Vt),Load (Pt),Temperature (Tt),Humidity (Ht),Wind Speed (Wt)
0,2024-01-01 00:00:00,0.591373,0.677601,0.597218,0.770155,0.265735
1,2024-01-01 01:00:00,0.474182,0.247547,0.725812,0.694685,0.519658
2,2024-01-01 02:00:00,0.619236,0.267982,0.387446,0.609627,0.045559
3,2024-01-01 03:00:00,0.780788,0.603131,0.934753,0.659472,0.083293
4,2024-01-01 04:00:00,0.456485,0.481089,0.132750,0.545043,0.285374


In [ ]:
features = ['Voltage (Vt)', 'Load (Pt)', 'Temperature (Tt)', 'Humidity (Ht)', 'Wind Speed (Wt)']
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])
df.head()


,Timestamp,Voltage (Vt),Load (Pt),Temperature (Tt),Humidity (Ht),Wind Speed (Wt)
0,2024-01-01 00:00:00,0.591373,0.677601,0.597218,0.770155,0.265735
1,2024-01-01 01:00:00,0.474182,0.247547,0.725812,0.694685,0.519658
2,2024-01-01 02:00:00,0.619236,0.267982,0.387446,0.609627,0.045559
3,2024-01-01 03:00:00,0.780788,0.603131,0.934753,0.659472,0.083293
4,2024-01-01 04:00:00,0.456485,0.481089,0.132750,0.545043,0.285374


In [ ]:
import numpy as np  # Add this if not already imported

def create_sequences(data, lookback=24, forecast_horizon=1):
    X, y = [], []
    for i in range(len(data) - lookback - forecast_horizon + 1):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback, 0])  # Predicting Voltage only
    return np.array(X), np.array(y)

data = df[features].values
X, y = create_sequences(data, lookback=24, forecast_horizon=1)
print("Total samples created:", X.shape, y.shape)


Total samples created: (8615, 24, 5) (8615,)


In [ ]:
def create_sequences(data, lookback=24, forecast_horizon=1):
    X, y = [], []
    for i in range(len(data) - lookback - forecast_horizon + 1):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback, 0])  # Voltage prediction
    return np.array(X), np.array(y)

data = df[features].values
X, y = create_sequences(data, lookback=24, forecast_horizon=1)
print("Total samples created:", X.shape, y.shape)


Total samples created: (8615, 24, 5) (8615,)


In [ ]:
from sklearn.model_selection import train_test_split  # Ensure this line is included

# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

# Print shapes
print("X_train:", X_train.shape)
print("X_val:  ", X_val.shape)
print("X_test: ", X_test.shape)


X_train: (6030, 24, 5)
X_val:   (1292, 24, 5)
X_test:  (1293, 24, 5)


In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

print("X_train:", X_train.shape)
print("X_val:", X_val.shape)
print("X_test:", X_test.shape)


X_train: (6030, 24, 5)
X_val: (1292, 24, 5)
X_test: (1293, 24, 5)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the LSTM model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(1)  # Predicting voltage
])

# Compile the model
model.compile(optimizer='adam', loss='mse')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Optional: early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 12s 89ms/step - loss: 0.0534 - val_loss: 0.0353
Epoch 2/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 9s 80ms/step - loss: 0.0360 - val_loss: 0.0363
Epoch 3/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 76ms/step - loss: 0.0361 - val_loss: 0.0354
Epoch 4/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 84ms/step - loss: 0.0348 - val_loss: 0.0351
Epoch 5/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 8s 83ms/step - loss: 0.0342 - val_loss: 0.0348
Epoch 6/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - loss: 0.0346 - val_loss: 0.0349
Epoch 7/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - loss: 0.0342 - val_loss: 0.0351
Epoch 8/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 82ms/step - loss: 0.0328 - val_loss: 0.0347
Epoch 9/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - loss: 0.0334 - val_loss: 0.0346
Epoch 10/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 72ms/step - loss: 0.0342 - val_loss: 0.0349
Epoch 11/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 71ms/step - loss: 0.0338 - val_loss: 0.0348
Epoch 12/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 70

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define the LSTM model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - loss: 0.0588 - val_loss: 0.0356
Epoch 2/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 8s 85ms/step - loss: 0.0358 - val_loss: 0.0353
Epoch 3/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - loss: 0.0355 - val_loss: 0.0368
Epoch 4/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - loss: 0.0362 - val_loss: 0.0349
Epoch 5/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 82ms/step - loss: 0.0345 - val_loss: 0.0355
Epoch 6/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 8s 86ms/step - loss: 0.0347 - val_loss: 0.0356
Epoch 7/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - loss: 0.0346 - val_loss: 0.0346
Epoch 8/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 71ms/step - loss: 0.0342 - val_loss: 0.0354
Epoch 9/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 71ms/step - loss: 0.0346 - val_loss: 0.0348
Epoch 10/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 81ms/step - loss: 0.0337 - val_loss: 0.0349
Epoch 11/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 8s 86ms/step - loss: 0.0334 - val_loss: 0.0348
Epoch 12/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 71m

In [ ]:
# ✅ Import libraries (if not already done)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ✅ Define the model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(1)
])

# ✅ Compile the model
model.compile(optimizer='adam', loss='mse')

# ✅ Setup early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# ✅ Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 12s 83ms/step - loss: 0.0507 - val_loss: 0.0373
Epoch 2/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 8s 86ms/step - loss: 0.0360 - val_loss: 0.0361
Epoch 3/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - loss: 0.0352 - val_loss: 0.0357
Epoch 4/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - loss: 0.0352 - val_loss: 0.0369
Epoch 5/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - loss: 0.0341 - val_loss: 0.0359
Epoch 6/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 79ms/step - loss: 0.0345 - val_loss: 0.0348
Epoch 7/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 86ms/step - loss: 0.0329 - val_loss: 0.0351
Epoch 8/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 84ms/step - loss: 0.0345 - val_loss: 0.0347
Epoch 9/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - loss: 0.0341 - val_loss: 0.0346
Epoch 10/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 8s 86ms/step - loss: 0.0345 - val_loss: 0.0346
Epoch 11/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - loss: 0.0330 - val_loss: 0.0348
Epoch 12/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 70

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define LSTM model architecture
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(1)  # Predict voltage
])

# Compile model
model.compile(optimizer='adam', loss='mse')

# Early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val)_


SyntaxError: '(' was never closed (ipython-input-14-384501626.py, line 21)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

# Predict on test set
y_pred = model.predict(X_test).flatten()

# Metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2 = r2_score(y_test, y_pred)

print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"R²:   {r2:.4f}")

# Plot predictions vs actual
plt.figure(figsize=(14,5))
plt.plot(y_test[:100], label='Actual Voltage')
plt.plot(y_pred[:100], label='Predicted Voltage')
plt.title("Actual vs Predicted Voltage (First 100 Test Samples)")
plt.xlabel("Time Step")
plt.ylabel("Voltage (Normalized)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot training & validation loss
plt.figure(figsize=(10,4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("MSE Loss")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Save model
model.save("GridForecast_LSTM_Model.h5")

# Optionally save scaler
import joblib
joblib.dump(scaler, "MinMaxScaler.save")


In [ ]:
from google.colab import files

# Download model
files.download("GridForecast_LSTM_Model.h5")

# Download scaler
files.download("MinMaxScaler.save")


In [ ]:
from google.colab import files

# Download model
files.download("GridForecast_LSTM_Model.h5")

# Download scaler
files.download("MinMaxScaler.save")


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

# Predict on test set
y_pred = model.predict(X_test).flatten()

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2 = r2_score(y_test, y_pred)

print(f"📊 Model Performance on Test Set:")
print(f"MAE  = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")
print(f"MAPE = {mape:.2f}%")
print(f"R²    = {r2:.4f}")

# Plot actual vs predicted voltage
plt.figure(figsize=(14, 5))
plt.plot(y_test[:100], label='Actual Voltage')
plt.plot(y_pred[:100], label='Predicted Voltage')
plt.title("🔍 Actual vs Predicted Voltage (First 100 Samples)")
plt.xlabel("Time Step")
plt.ylabel("Normalized Voltage")
plt.legend()
plt.grid(True)
plt.show()

# Plot training/validation loss
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("📉 Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

# Predict on test set
y_pred = model.predict(X_test).flatten()

# Evaluation Metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2 = r2_score(y_test, y_pred)

# 📊 Print Results
print("📊 Final Evaluation Metrics on Test Set:")
print(f"MAE   = {mae:.4f}")
print(f"RMSE  = {rmse:.4f}")
print(f"MAPE  = {mape:.2f}%")
print(f"R²     = {r2:.4f}")

# 📈 Plot: Actual vs Predicted Voltage
plt.figure(figsize=(14, 5))
plt.plot(y_test[:100], label='Actual Voltage')
plt.plot(y_pred[:100], label='Predicted Voltage')
plt.title("🔍 Actual vs Predicted Voltage (First 100 Samples)")
plt.xlabel("Time Step")
plt.ylabel("Voltage (Normalized)")
plt.legend()
plt.grid(True)
plt.show()

# 📉 Plot: Training vs Validation Loss
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("📉 Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from google.colab import files

# Predict on test set
y_pred = model.predict(X_test).flatten()

# Metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2 = r2_score(y_test, y_pred)

print("📊 Final Evaluation Metrics on Test Set:")
print(f"MAE   = {mae:.4f}")
print(f"RMSE  = {rmse:.4f}")
print(f"MAPE  = {mape:.2f}%")
print(f"R²     = {r2:.4f}")

# Plot 1
plt.figure(figsize=(14, 5))
plt.plot(y_test[:100], label='Actual Voltage', linewidth=2)
plt.plot(y_pred[:100], label='Predicted Voltage', linestyle='--')
plt.title("🔍 Actual vs Predicted Voltage (First 100 Samples)")
plt.xlabel("Time Step")
plt.ylabel("Voltage (Normalized)")
plt.legend()
plt.grid(True)
plt.savefig("actual_vs_predicted.png")
plt.show()

# Plot 2
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("📉 Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.legend()
plt.grid(True)
plt.savefig("loss_curve.png")
plt.show()

# Downloads
files.download("actual_vs_predicted.png")
files.download("loss_curve.png")


In [ ]:
plt.savefig("actual_vs_predicted.png")
plt.show()

...

plt.savefig("loss_curve.png")
plt.show()


In [ ]:
files.download("actual_vs_predicted.png")
files.download("loss_curve.png")
